In [1]:
import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

if tf.test.gpu_device_name():
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
   print("Please install GPU version of TF")

from model.YOLOv4 import YOLOv4
import numpy as np
import gc
from tqdm import tqdm

SIZE = 320
BATCH_SIZE = 4
network = YOLOv4(side=SIZE)
from dataset import Dataset
dataset = Dataset(BATCH_SIZE, network.layers[-1].output_shape, shape=network.layers[0].input_shape[1: 3])
print("input shape is :", dataset.shape)

image_path = dataset.get_dir()
labels = dataset.get_label_data()
labels = tf.ragged.constant(labels)

Default GPU Device:/device:GPU:0
input shape is : (320, 320)


KeyboardInterrupt: 

In [ ]:
train_dataloader = tf.data.Dataset.from_tensor_slices((image_path, labels))
train_dataloader = train_dataloader.map(dataset.preprocess_X_y, num_parallel_calls = tf.data.experimental.AUTOTUNE)
train_dataloader = train_dataloader.shuffle(buffer_size = 1024)
train_dataloader = train_dataloader.batch(batch_size = BATCH_SIZE)
train_dataloader = train_dataloader.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

from loss import YOLO_loss

output_layer_1 = [layer for layer in network.layers[-1].layers if layer.name == 'output_1'][0]
output_layer_2 = [layer for layer in network.layers[-1].layers if layer.name == 'output_2'][0]
network.compile(loss={'output_3': YOLO_loss(dataset, output_layer_1.output, output_layer_2.output)}, optimizer="adam", metrics="acc")
network.build((None, SIZE, SIZE, 3))

network.summary()
#tf.keras.utils.plot_model(network.model(), show_shapes=True, expand_nested=True, dpi=300)

network.fit(train_dataloader)